In [4]:
import os
import numpy as np
from random import seed
from keras.models import load_model
from keras import backend as K
from time import time
import pandas as pd
from datetime import timedelta

#home_dir = '/home/rick/Documents/HannahZ'
#code_dir = home_dir + '/Code'
#model_dir = code_dir + '/Final_models'
#os.chdir(home_dir)

# on my computer (windows):
# where to find the images:
home_dir = 'C:\\Users\\ThinkPad User\\Google Drive\\DeepSolaris'
# # folder where to save code and so on
code_dir = 'C:\\Users\\ThinkPad User\\Google Drive\\Master_thesis_H_Z_DL&NN\\Results'
# # where to save the final models
model_dir = 'E:\\FinalModels\\'
# # where to save the logs for visualization in TensorBoard
log_dir = code_dir + '\\logs'

os.chdir(home_dir)
train_images = np.load('training_images_AcMüDüHo.npy')
valid_images = np.load('validation_images_AcMüDüHo.npy')
test_images = np.load('test_images_AcMüDüHo.npy')
train_labels = np.load('training_labels_AcMüDüHo.npy')
valid_labels = np.load('validation_labels_AcMüDüHo.npy')
test_labels = np.load('test_labels_AcMüDüHo.npy')

print("Size of:")
print("- Training-set:\t\t{}".format(len(train_labels)))
print("- Validation-set:\t{}".format(len(valid_labels)))
print("- Test-set:\t\t{}".format(len(test_labels)))

Size of:
- Training-set:		3357
- Validation-set:	374
- Test-set:		934


In [5]:
seed(0)
### Helper metrics functions
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def fbeta_score(y_true, y_pred, beta=1):
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')
    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

def fmeasure(y_true, y_pred):
    return fbeta_score(y_true, y_pred, beta=1)

In [6]:
os.chdir(model_dir)
nameList = os.listdir(model_dir)
#
for el in nameList:
    print(el)

DenseNet121_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0002_e-48_imagenet_141_2-e30
DenseNet121_bs-64_triangular_sz-315_blr-1e-05_mlr-0.0009_e-48_imagenet_Frozen_3-e04
DenseNet121_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0003_e-96_imagenet_All_2-e08
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0001_e-48_imagenet_595_3-e29
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0003_e-96_imagenet_All_3-e63
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0005_e-48_imagenet_Frozen_4-e16
IncV3_bs-64_triangular2_sz-315_blr-1e-05_mlr-0.0002_e-96_imagenet_All_0-e04
IncV3_bs-64_triangular2_sz-315_blr-1e-06_mlr-0.0001_e-96_imagenet_Frozen_4-e08
IncV3_bs-64_triangular2_sz-420_blr-1e-06_mlr-0.0002_e-96_imagenet_229__ensemble__
Xception_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0002_e-48_imagenet_Frozen_3-e42
Xception_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0004_e-96_imagenet_All_1-e73
Xception_bs-64_triangular_sz-315_blr-5e-05_mlr-0.0006_e-48_imagenet_86_3-e19


In [8]:
os.chdir(model_dir)
def evalModels():
    start_time = time()
    global test_acc_list; global valid_acc_list; global train_acc_list; global test_loss_list
    global valid_loss_list; global train_loss_list; global test_prec_list; global valid_prec_list
    global train_prec_list; global test_rec_list; global valid_rec_list; global train_rec_list
    global model_details
    model_details = []; test_acc_list = []; valid_acc_list = []; train_acc_list = []; test_loss_list = []
    valid_loss_list = []; train_loss_list = []; test_prec_list = []; valid_prec_list = []
    train_prec_list = []; test_rec_list = []; valid_rec_list = []; train_rec_list = []
    for name in nameList:
        print(name)
        Final_model = load_model(name, custom_objects={'precision': precision, 'recall': recall, 
                                 'fbeta_score': fbeta_score, 'fmeasure': fmeasure})
        train_metrics = Final_model.evaluate(train_images, train_labels)
        valid_metrics = Final_model.evaluate(valid_images, valid_labels)
        test_metrics = Final_model.evaluate(test_images, test_labels)
        train_acc = train_metrics[1]; valid_acc = valid_metrics[1]; test_acc = test_metrics[1]
        train_loss = train_metrics[0]; valid_loss = valid_metrics[0]; test_loss = test_metrics[0]
        train_prec = train_metrics[2];  valid_prec = valid_metrics[2]; test_prec = test_metrics[2]
        train_rec = train_metrics[3]; valid_rec = valid_metrics[3]; test_rec = test_metrics[3]         
        test_acc_list.append(test_acc); valid_acc_list.append(valid_acc); train_acc_list.append(train_acc)
        test_loss_list.append(test_loss); valid_loss_list.append(valid_loss); train_loss_list.append(train_loss)
        test_prec_list.append(test_prec); valid_prec_list.append(valid_prec); train_prec_list.append(train_prec)
        test_rec_list.append(test_rec); valid_rec_list.append(valid_rec); train_rec_list.append(train_rec)
        model_details.append(name)
    end_time = time()
    time_dif = end_time - start_time
    print('Time usage: ' + str(timedelta(seconds=int(round(time_dif)))) + ' minutes')
evalModels()

IncV3_bs-64_triangular2_sz-420_blr-1e-06_mlr-0.0002_e-96_imagenet_229__ensemble__
934/934 [==============================] - 1s 1ms/step
IncV3_bs-64_triangular2_sz-315_blr-1e-06_mlr-0.0001_e-96_imagenet_Frozen_4-e08
934/934 [==============================] - 1s 1000us/step
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0005_e-48_imagenet_Frozen_4-e16
934/934 [==============================] - 2s 2ms/step
Xception_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0002_e-48_imagenet_Frozen_3-e42
934/934 [==============================] - 2s 2ms/step
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0003_e-96_imagenet_All_3-e63
934/934 [==============================] - 2s 2ms/step
Xception_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0004_e-96_imagenet_All_1-e73
934/934 [==============================] - 1s 2ms/step
DenseNet121_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0002_e-48_imagenet_141_2-e30
934/934 [==============================] - 1s 1ms/step
DenseNet121_bs-64_triangular_sz-315_blr-1e-

NameError: global name 'timedelta' is not defined

In [9]:
columns = ['test acc', 'valid acc', 'train acc', 'test loss', 'valid loss', 'train loss', 'test prec', \
           'valid prec', 'train prec', 'test rec', 'valid rec', 'train rec']
data = np.array((test_acc_list, valid_acc_list, train_acc_list, test_loss_list, valid_loss_list, train_loss_list,\
                test_prec_list, valid_prec_list, train_prec_list, test_rec_list, valid_rec_list, train_rec_list))
results = pd.DataFrame(data = data, columns = model_details, index = columns)
AllModel_overview = pd.DataFrame(data = results.transpose(), columns = columns, index = model_details)
AllModel_overview

,test acc,valid acc,train acc,test loss,valid loss,train loss,test prec,valid prec,train prec,test rec,valid rec,train rec
IncV3_bs-64_triangular2_sz-420_blr-1e-06_mlr-0.0002_e-96_imagenet_229__ensemble__,0.745182,0.748663,0.769139,0.511855,0.531159,0.491557,0.741130,0.710675,0.754507,0.773061,0.783557,0.792240
IncV3_bs-64_triangular2_sz-315_blr-1e-06_mlr-0.0001_e-96_imagenet_Frozen_4-e08,0.721627,0.716578,0.739053,0.568132,0.600492,0.539740,0.716291,0.686933,0.728786,0.758488,0.741045,0.751967
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0005_e-48_imagenet_Frozen_4-e16,0.804069,0.794118,0.812630,0.608706,0.692758,0.559599,0.823069,0.804819,0.825622,0.781818,0.723480,0.785323
Xception_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0002_e-48_imagenet_Frozen_3-e42,0.795503,0.732620,0.793566,1.078451,1.337860,1.029621,0.736208,0.666597,0.737768,0.943786,0.844355,0.912541
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0003_e-96_imagenet_All_3-e63,0.794433,0.775401,0.806971,0.814526,0.933897,0.716140,0.757914,0.743769,0.772465,0.875015,0.799753,0.867392
Xception_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0004_e-96_imagenet_All_1-e73,0.820128,0.764706,0.810843,1.235605,1.607674,1.256852,0.777590,0.730212,0.779639,0.908454,0.767969,0.865049
DenseNet121_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0002_e-48_imagenet_141_2-e30,0.771949,0.791444,0.811141,0.895999,0.815620,0.686062,0.731626,0.716107,0.756181,0.883377,0.932906,0.915417
DenseNet121_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0003_e-96_imagenet_All_2-e08,0.789079,0.802139,0.825439,0.725991,0.662451,0.552517,0.749822,0.724462,0.775477,0.888218,0.931279,0.917116
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0001_e-48_imagenet_595_3-e29,0.781585,0.764706,0.800119,0.650173,0.751039,0.616011,0.757196,0.730710,0.771101,0.838072,0.779671,0.851351
DenseNet121_bs-64_triangular_sz-315_blr-1e-05_mlr-0.0009_e-48_imagenet_Frozen_3-e04,0.800857,0.799465,0.827525,0.584331,0.520815,0.447890,0.786946,0.754608,0.807499,0.843552,0.865484,0.857473


In [10]:
testData = np.array((test_acc_list, test_loss_list, test_prec_list, test_rec_list))
testColumns = ['test acc', 'test loss', 'test prec', 'test rec']
shortRes = pd.DataFrame(data = testData, columns = model_details, index = testColumns)
ShortAllModelOvw = pd.DataFrame(data = shortRes.transpose(), columns = testColumns, index = model_details)
ShortAllModelOvw.sort_values(by=['test acc'], ascending = False)

,test acc,test loss,test prec,test rec
Xception_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0004_e-96_imagenet_All_1-e73,0.820128,1.235605,0.777590,0.908454
Xception_bs-64_triangular_sz-315_blr-5e-05_mlr-0.0006_e-48_imagenet_86_3-e19,0.814775,0.740570,0.822492,0.808958
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0005_e-48_imagenet_Frozen_4-e16,0.804069,0.608706,0.823069,0.781818
DenseNet121_bs-64_triangular_sz-315_blr-1e-05_mlr-0.0009_e-48_imagenet_Frozen_3-e04,0.800857,0.584331,0.786946,0.843552
Xception_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0002_e-48_imagenet_Frozen_3-e42,0.795503,1.078451,0.736208,0.943786
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0003_e-96_imagenet_All_3-e63,0.794433,0.814526,0.757914,0.875015
DenseNet121_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0003_e-96_imagenet_All_2-e08,0.789079,0.725991,0.749822,0.888218
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0001_e-48_imagenet_595_3-e29,0.781585,0.650173,0.757196,0.838072
DenseNet121_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0002_e-48_imagenet_141_2-e30,0.771949,0.895999,0.731626,0.883377
IncV3_bs-64_triangular2_sz-420_blr-1e-06_mlr-0.0002_e-96_imagenet_229__ensemble__,0.745182,0.511855,0.741130,0.773061


In [35]:
os.chdir(code_dir + '/Excel')
AllModel_overview.to_csv('AllModelOverview')

In [19]:
os.chdir(home_dir)
Htrain_images = np.load('train_images_CBS.npy')
Htest_images = np.load('test_images_CBS.npy')
Htrain_labels = np.load('train_labels_CBS.npy')
Htest_labels = np.load('test_labels_CBS.npy')

print("Size of:")
print("- Training-set:\t\t{}".format(len(Htrain_labels)))
print("- Test-set:\t\t{}".format(len(Htest_labels)))

Size of:
- Training-set:		1848
- Test-set:		464


In [20]:
os.chdir(model_dir)
def evalModels():
    start_time = time()
    global Htest_acc_list; global Htrain_acc_list; global Htest_loss_list
    global Htrain_loss_list; global Htest_prec_list; 
    global Htrain_prec_list; global Htest_rec_list; global Htrain_rec_list
    global model_details
    model_details = []; Htest_acc_list = []; Htrain_acc_list = []; Htest_loss_list = []
    Htrain_loss_list = []; Htest_prec_list = []; 
    Htrain_prec_list = []; Htest_rec_list = []; Htrain_rec_list = []
    for name in nameList:
        print(name)
        Final_model = load_model(name, custom_objects={'precision': precision, 'recall': recall, 
                                 'fbeta_score': fbeta_score, 'fmeasure': fmeasure})
        train_metrics = Final_model.evaluate(Htrain_images, Htrain_labels)
        test_metrics = Final_model.evaluate(Htest_images, Htest_labels)
        train_acc = train_metrics[1]; test_acc = test_metrics[1]
        train_loss = train_metrics[0]; test_loss = test_metrics[0]
        train_prec = train_metrics[2];  test_prec = test_metrics[2]
        train_rec = train_metrics[3]; test_rec = test_metrics[3]         
        Htest_acc_list.append(test_acc); Htrain_acc_list.append(train_acc)
        Htest_loss_list.append(test_loss); Htrain_loss_list.append(train_loss)
        Htest_prec_list.append(test_prec); Htrain_prec_list.append(train_prec)
        Htest_rec_list.append(test_rec); Htrain_rec_list.append(train_rec)
        model_details.append(name)
    end_time = time()
    time_dif = end_time - start_time
    print('Time usage: ' + str(timedelta(seconds=int(round(time_dif)))) + ' minutes')
evalModels()

IncV3_bs-64_triangular2_sz-420_blr-1e-06_mlr-0.0002_e-96_imagenet_229__ensemble__
464/464 [==============================] - 1s 2ms/step
IncV3_bs-64_triangular2_sz-315_blr-1e-06_mlr-0.0001_e-96_imagenet_Frozen_4-e08
464/464 [==============================] - 1s 1ms/step
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0005_e-48_imagenet_Frozen_4-e16
464/464 [==============================] - 1s 2ms/step
Xception_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0002_e-48_imagenet_Frozen_3-e42
464/464 [==============================] - 1s 2ms/step
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0003_e-96_imagenet_All_3-e63
464/464 [==============================] - 1s 2ms/step
Xception_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0004_e-96_imagenet_All_1-e73
464/464 [==============================] - 1s 2ms/step
DenseNet121_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0002_e-48_imagenet_141_2-e30
464/464 [==============================] - 1s 2ms/step
DenseNet121_bs-64_triangular_sz-315_blr-1e-06_

NameError: global name 'timedelta' is not defined

In [21]:
print('    HEERLEN    ')
testData = np.array((Htest_acc_list, Htest_loss_list, Htest_prec_list, Htest_rec_list))
testColumns = ['test acc', 'test loss', 'test prec', 'test rec']
shortRes = pd.DataFrame(data = testData, columns = model_details, index = testColumns)
ShortAllModelOvw = pd.DataFrame(data = shortRes.transpose(), columns = testColumns, index = model_details)
ShortAllModelOvw.sort_values(by=['test acc'], ascending = False)

    HEERLEN    


,test acc,test loss,test prec,test rec
DenseNet121_bs-64_triangular_sz-315_blr-1e-05_mlr-0.0009_e-48_imagenet_Frozen_3-e04,0.620690,0.928866,0.509031,0.510776
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0005_e-48_imagenet_Frozen_4-e16,0.590517,0.933536,0.510345,0.415948
IncV3_bs-64_triangular2_sz-420_blr-1e-06_mlr-0.0002_e-96_imagenet_229__ensemble__,0.549569,0.709604,0.500750,0.370690
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0001_e-48_imagenet_595_3-e29,0.543103,1.494908,0.501784,0.517241
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0003_e-96_imagenet_All_3-e63,0.540948,1.827066,0.503193,0.504310
Xception_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0002_e-48_imagenet_Frozen_3-e42,0.538793,1.697675,0.501784,0.538793
Xception_bs-64_triangular_sz-315_blr-5e-05_mlr-0.0006_e-48_imagenet_86_3-e19,0.538793,1.662179,0.501784,0.536638
Xception_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0004_e-96_imagenet_All_1-e73,0.534483,3.167065,0.501149,0.547414
IncV3_bs-64_triangular2_sz-315_blr-1e-06_mlr-0.0001_e-96_imagenet_Frozen_4-e08,0.532328,0.750419,0.499179,0.318966
DenseNet121_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0002_e-48_imagenet_141_2-e30,0.530172,2.503577,0.500556,0.549569


In [22]:
os.chdir(home_dir)
Btrain_images = np.load('train_images_Bradbury.npy')
Btest_images = np.load('test_images_Bradbury.npy')
Btrain_labels = np.load('train_labels_Bradbury.npy')
Btest_labels = np.load('test_labels_Bradbury.npy')

print("Size of:")
print("- Training-set:\t\t{}".format(len(Btrain_labels)))
print("- Test-set:\t\t{}".format(len(Btest_labels)))

Size of:
- Training-set:		31776
- Test-set:		7946


In [23]:
os.chdir(model_dir)
def evalModels():
    start_time = time()
    global Btest_acc_list; global Btrain_acc_list; global Btest_loss_list
    global Btrain_loss_list; global Btest_prec_list; 
    global Btrain_prec_list; global Btest_rec_list; global Btrain_rec_list
    global model_details
    model_details = []; Btest_acc_list = []; Btrain_acc_list = []; Btest_loss_list = []
    Btrain_loss_list = []; Btest_prec_list = []; 
    Btrain_prec_list = []; Btest_rec_list = []; Btrain_rec_list = []
    for name in nameList:
        print(name)
        Final_model = load_model(name, custom_objects={'precision': precision, 'recall': recall, 
                                 'fbeta_score': fbeta_score, 'fmeasure': fmeasure})
        train_metrics = Final_model.evaluate(Btrain_images, Btrain_labels)
        test_metrics = Final_model.evaluate(Btest_images, Btest_labels)
        train_acc = train_metrics[1]; test_acc = test_metrics[1]
        train_loss = train_metrics[0]; test_loss = test_metrics[0]
        train_prec = train_metrics[2];  test_prec = test_metrics[2]
        train_rec = train_metrics[3]; test_rec = test_metrics[3]         
        Btest_acc_list.append(test_acc); Btrain_acc_list.append(train_acc)
        Btest_loss_list.append(test_loss); Btrain_loss_list.append(train_loss)
        Btest_prec_list.append(test_prec); Btrain_prec_list.append(train_prec)
        Btest_rec_list.append(test_rec); Btrain_rec_list.append(train_rec)
        model_details.append(name)
    end_time = time()
    time_dif = end_time - start_time
    print('Time usage: ' + str(timedelta(seconds=int(round(time_dif)))) + ' minutes')
evalModels()

IncV3_bs-64_triangular2_sz-420_blr-1e-06_mlr-0.0002_e-96_imagenet_229__ensemble__
7946/7946 [==============================] - 14s 2ms/step
IncV3_bs-64_triangular2_sz-315_blr-1e-06_mlr-0.0001_e-96_imagenet_Frozen_4-e08
7946/7946 [==============================] - 14s 2ms/step
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0005_e-48_imagenet_Frozen_4-e16
7946/7946 [==============================] - 22s 3ms/step
Xception_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0002_e-48_imagenet_Frozen_3-e42
7946/7946 [==============================] - 19s 2ms/step
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0003_e-96_imagenet_All_3-e63
7946/7946 [==============================] - 23s 3ms/step
Xception_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0004_e-96_imagenet_All_1-e73
7946/7946 [==============================] - 19s 2ms/step
DenseNet121_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0002_e-48_imagenet_141_2-e30
7946/7946 [==============================] - 17s 2ms/step
DenseNet121_bs-64_triangu

NameError: global name 'timedelta' is not defined

In [24]:
print('    CALIFORNIA - BRADBURY    ')
BtestData = np.array((Btest_acc_list, Btest_loss_list, Btest_prec_list, Btest_rec_list))
testColumns = ['test acc', 'test loss', 'test prec', 'test rec']
shortRes = pd.DataFrame(data = BtestData, columns = model_details, index = testColumns)
ShortAllModelOvw = pd.DataFrame(data = shortRes.transpose(), columns = testColumns, index = model_details)
ShortAllModelOvw.sort_values(by=['test acc'], ascending = False)

    CALIFORNIA - BRADBURY    


,test acc,test loss,test prec,test rec
IncV3_bs-64_triangular2_sz-315_blr-1e-06_mlr-0.0001_e-96_imagenet_Frozen_4-e08,0.587088,5.887381,0.500042,0.344349
DenseNet121_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0003_e-96_imagenet_All_2-e08,0.584067,5.435911,0.499967,0.493028
DenseNet121_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0002_e-48_imagenet_141_2-e30,0.572112,5.510268,0.499946,0.484596
DenseNet121_bs-64_triangular_sz-315_blr-1e-05_mlr-0.0009_e-48_imagenet_Frozen_3-e04,0.556381,4.881508,0.500103,0.368814
IncV3_bs-64_triangular2_sz-315_blr-1e-05_mlr-0.0002_e-96_imagenet_All_0-e04,0.500126,7.968811,0.500000,0.503398
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0005_e-48_imagenet_Frozen_4-e16,0.500000,8.059048,0.000000,0.000000
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0003_e-96_imagenet_All_3-e63,0.500000,8.059048,0.000000,0.000000
IncResNetV2_bs-64_triangular_sz-315_blr-1e-06_mlr-0.0001_e-48_imagenet_595_3-e29,0.500000,8.059048,0.000000,0.000000
IncV3_bs-64_triangular2_sz-420_blr-1e-06_mlr-0.0002_e-96_imagenet_229__ensemble__,0.496980,5.251459,0.500000,0.497105
Xception_bs-128_triangular_sz-157_blr-1e-06_mlr-0.0002_e-48_imagenet_Frozen_3-e42,0.496980,7.833992,0.500000,0.493833


In [25]:
os.chdir(code_dir)
import DenseNet_pretrained
from DenseNet_pretrained import DenseNet
from DenseNet_pretrained import conv_block
from DenseNet_pretrained import transition_block
from DenseNet_pretrained import dense_block
from DenseNet_pretrained import DenseNet121

In [7]:
from keras import backend as K
from keras.models import Model
from keras.layers import Activation
from keras.layers import AveragePooling2D
from keras.layers import BatchNormalization
from keras.layers import Concatenate
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import ZeroPadding2D
from keras.utils.data_utils import get_file
from keras.engine import get_source_inputs
from keras.applications import imagenet_utils
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import _obtain_input_shape 


In [42]:
from keras.layers import Input
input_shape = test_images[0,:,:,:].shape
model_input = Input(shape=input_shape)

In [43]:
def compileModel(Model_base, trainable_layers = 'Frozen'):
    global Final_model
    x = Model_base.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(512, activation='relu')(x)
    preds = Dense(1, activation='sigmoid')(x)
    Final_model = Model(inputs=Model_base.input, outputs=preds)
    if (trainable_layers == 'Frozen'):
        for layer in Model_base.layers:   # only train the new layers first
            layer.trainable = False
    elif (trainable_layers == 'All'):
        for layer in Model_base.layers:   # only train the new layers first
            layer.trainable = True
    else:
        for layer in Model_base.layers[:trainable_layers]:
            layer.trainable = False
        for layer in Model_base.layers[trainable_layers:]:  
            layer.trainable = True
    Final_model.compile(optimizer='rmsprop', loss='binary_crossentropy', 
                            metrics=['accuracy', precision, recall, fmeasure])

In [46]:
epochs = 48
base_lr = 0.000005
max_lr = 0.0002
mode = 'triangular'
batch_size = 64
step_size = 315
def train_model(verbose =1, times = 2, epochs = epochs, base_lr = base_lr, max_lr= max_lr, mode = mode, 
                stepsize = step_size, batch_size = batch_size, use_tensorboard = False):
    start_time = time()   
    global h
    global no_of_epochs
    global name
    date = str(datetime.now().date())
    # logdir in linux:
    logdir = (log_dir + '/' + date + '_' + str(batch_size) + '_' + str(mode) + '_si-' + str(stepsize) + '_base-' 
              + str(base_lr) + '_max-' + str(max_lr) + '_' + str(times))
    # logdir in windows:
    ##logdir = (log_dir + '\\' + date + '_' + str(batch_size) + '_' + str(mode) + '_si-' + str(stepsize) + 
    #          '_base-' + str(base_lr) + '_max-' + str(max_lr) + '_' + str(times))
    name = ('batch size: ' + str(batch_size) + ', mode: ' + str(mode) + ', stepsize: ' 
            + str(stepsize) + ', base-lr: ' + str(base_lr) + ', max-lr ' + str(max_lr) + ', *' + str(times))
    print(' - Model specifics:\t{}'.format(name))
    clr = CyclicLR(base_lr = base_lr, max_lr=max_lr,
                        step_size= stepsize, mode = mode)
    tbCallBack = TensorBoard(log_dir=logdir, histogram_freq=0, 
                         write_graph=False, write_images=False)
    if (use_tensorboard == True):
        Final_model.fit(x = train_images, y = train_labels, epochs = epochs, callbacks= [tbCallBack, clr], 
                            validation_data = (valid_images, valid_labels), batch_size = batch_size, 
                            verbose = verbose)
    else:
        Final_model.fit(x = train_images, y = train_labels, epochs = epochs, callbacks= [clr], 
                            validation_data = (valid_images, valid_labels), batch_size = batch_size, 
                            verbose = verbose)
    no_of_epochs = no_of_epochs + epochs
    h = []
    h = clr.history
    end_time = time()
    time_dif = end_time - start_time
    print('Time usage: ' + str(timedelta(seconds=int(round(time_dif)))) + ' minutes')

In [47]:
import keras
ModelBaseIncV3 = keras.applications.inception_v3.InceptionV3(weights = 'imagenet', include_top=False, 
                                                             input_tensor = model_input)
print('first done')
ModelBaseIncResNetV2 = keras.applications.inception_resnet_v2.InceptionResNetV2(weights = 'imagenet', 
                                                                                include_top=False, 
                                                                                input_tensor = model_input)
print('second done')
ModelBaseDenseNet121 = DenseNet_pretrained.DenseNet121(weights = 'imagenet', include_top=False, 
                                                      input_tensor = model_input)
print('third done')
ModelBaseXception = keras.applications.xception.Xception(weights = 'imagenet', include_top=False,
                                                        input_tensor = model_input)
print('fourth done')

first done
second done
third done
fourth done


In [48]:
# IncV3
compileModel(trainable_layers = 229, Model_base= ModelBaseIncV3)
train_model(verbose =1, times = 1, epochs = 48, base_lr = 0.000001, max_lr= 0.0002, mode = triangular2, 
                stepsize = 420, batch_size = 64, use_tensorboard = False)
boIncV3 = Final_model
print('IncV3 done')
# IncResNetV2
compileModel(trainable_layers = 'Frozen', Model_base= ModelBaseIncResNetV2)
train_model(verbose =1, times = 1, epochs = 48, base_lr = 0.000001, max_lr= 0.0005, mode = triangular, 
                stepsize = 315, batch_size = 64, use_tensorboard = False)
boIncResNetV2 = Final_model
print('IncResNetV2 done')
# DenseNet121
compileModel(trainable_layers = 'Frozen', Model_base= ModelBaseDenseNet121)
train_model(verbose = 1, times = 1, epochs = 48, base_lr = 0.00001, max_lr= 0.0009, mode = triangular, 
                stepsize = 315, batch_size = 64, use_tensorboard = False)
boDenseNet121 = Final_model
print('DenseNet121 done')
#Xception
compileModel(trainable_layers = 'All', Model_base= ModelBaseXception)
train_model(verbose = 1, times = 1, epochs = 48, base_lr = 0.000001, max_lr= 0.0004, mode = triangular, 
                stepsize = 157, batch_size = 128, use_tensorboard = False)
boXception = Final_model

NameError: name 'triangular2' is not defined

In [ ]:
from keras.models import Model

models = [boIncV3, boIncResNetV2, boDenseNet121, boXception]
BaseModels = [ModelBaseIncV3, ModelBaseIncResNetV2, ModelBaseDenseNet121, ModelBaseXception]
def ensemble(models):    
    global boModel
    outputs = [model.outputs[0] for model in models]
    y = keras.layers.Average()(outputs)   
    boModel = Model(inputs= model_input, outputs = y, name='ensemble')  
    boModel.compile(optimizer='rmsprop', loss='binary_crossentropy',
                   metrics=['accuracy', precision, recall, fmeasure])
    return boModel
boEnsembleModel = ensemble(models)

In [15]:
ModelDic = {}
modelsE = [boIncV3, boIncResNetV2, boDenseNet121, boXception, boEnsembleModel]
metrics = ['loss', 'acc', 'precision', 'recall']
for name in modelsE;
    FinalModel = load_model(name)
    Res = FinalModel.evaluate(test_images, test_labels)
    ModelDic["{}".format(name)]= pd.DataFrame(data = Res[:4], columns = [name], index = metrics)

FinalResults = pd.concat([ModelDic['boIncV3'], ModelDic['boIncResNetV2'], ModelDic['boDenseNet121'], 
                          ModelDic['boXception'], ModelDic['boEnsembleModel']], axis=1)
FinalResults = pd.DataFrame(data = FinalResults.transpose(), columns = metrics)
FinalResults.sort_values(by = ['acc'], ascending = False)

SyntaxError: invalid syntax (<ipython-input-15-ff0565607e0b>, line 4)

In [16]:
os.chdir(model_dir)
boEnsembleModel.save('bestOfModelsEnsemble')

NameError: name 'boEnsembleModel' is not defined